# Laboratorio 11

La finalidad de este laboratorio es tener un mejor manejo de las herramientas que nos ofrece Scikit-Learn, como los _transformers_ y _pipelines_.  Usaremos el dataset [The Current Population Survey (CPS)](https://www.openml.org/d/534) que consiste en predecir el salario de una persona en función de atributos como la educación, experiencia o edad.

In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

%matplotlib inline

Como siempre, un pequeño análisis descriptivo

In [ ]:
survey = fetch_openml(data_id=534, as_frame=True)

In [ ]:
X = survey.data[survey.feature_names]
X.head()

In [ ]:
X.describe(include="all").T.fillna("")

In [ ]:
y = survey.target
y.head()

Y la posterior partición _train/test_.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42
)

## Ejercicio 1

(1 pto)

_One-Hot Encode_ es una técnica que a partir de una _feature_ categórica generar múltiples columnas, una por categoría.

* Define el transformador `ohe_sex` utilizando `OneHotEncoder` con atributos `drop="if_binary"` y `sparse=False`, luego ajusta y transforma el dataframe `X` solo con la columna `SEX`.
* Define el transformador `ohe_race` utilizando `OneHotEncoder` con atributos `drop="if_binary"` y `sparse=False`, luego ajusta y transforma el dataframe `X` solo con la columna `RACE`.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
ohe_sex = OneHotEncoder(drop = 'if_binary', sparse = False)
ohe_sex.fit_transform(X['SEX'].to_frame())

In [ ]:
ohe_race = OneHotEncoder(drop = 'if_binary', sparse = False)
ohe_race.fit_transform(X['RACE'].to_frame())

__Pregunta:__ ¿Por qué las transformaciones resultantes tiene diferente cantidad de columnas?

__Respuesta:__ Porque tienen distintas cantidades de categorías, y la cantidad de columnas se corresponde con la cantidad de categorías (a excepción de las categorías binarias, donde solo hay una columna ya que esta contiene toda la información).

## Ejercicio 2

(1 pto)

Realizar _One-Hot-Encoding_ para cada una de las columnas categóricas y luego unirlas en un nuevo array o dataframe es tedioso, poco escablable y probablemente conlleve a errores. La función `make_column_transformer` permite automatizar este proceso en base a aplicar transformadores a distintas columnas.

* `categorical_columns` debe ser una lista con todos los nombres de columnas categóricas del dataframe `X`.
* `numerical_columns` debe ser una lista con todos los nombres de columnas numéricas del dataframe `X`.
* Define `preprocessor` utilizando `make_column_transformer` tal que:
    - A las columnas categóricas se les aplique `OneHotEncoder` con el argumento `drop="if_binary"`
    - El resto de las columnas se mantenga igual. Hint: Revisar la documentación del argumento `remainder`.
* Finalmente define  `X_processed` al ajustar y transformar el dataframe `X` utilizando `preprocessor` 

In [ ]:
from sklearn.compose import make_column_transformer

categorical_columns = ['SOUTH', 'SEX', 'UNION', 'RACE', 'OCCUPATION', 'SECTOR', 'MARR']
numerical_columns = ['EDUCATION', 'EXPIRIENCE', 'AGE']

preprocessor = make_column_transformer(
    (OneHotEncoder(drop='if_binary'),categorical_columns),
    remainder='passthrough'
)

X_processed = preprocessor.fit_transform(X)
print(X_processed)

In [ ]:
print(f"X_processed tiene {X_processed.shape[0]} filas y {X_processed.shape[1]} columnas.")

## Ejercicio 3

(1 pto)

Sucede un fenómeno similar al aplicar transformaciones al vector de respuesta. En ocasiones es necesario transformarlo pero que las predicciones sean en la misma escala original. `TransformedTargetRegressor` juega un rol clave, pues los insumos necesarios son: un estimador, la función y la inversa para aplicar al vector de respuesta.

Define `ttr` como un `TransformedTargetRegressor` tal que:
* El regresor sea un modelo de regresión Ridge y parámetro de regularización `1e-10`.
* La función para transformar sea logaritmo base 10. Hint: `NumPy` es tu amigo.
* La función inversa sea aplicar `10**x`. Hint: Revisa el módulo `special` de `SciPy` en la sección de _Convenience functions_.

In [ ]:
from sklearn.compose import TransformedTargetRegressor
from sklearn.linear_model import Ridge

In [ ]:
ttr = TransformedTargetRegressor(
    regressor = Ridge(alpha = 1e-10),
    func = np.log10,
    inverse_func = sp.special.exp10
    )

Ajusta el modelo con los datos de entrenamiento

In [ ]:
ttr.fit(X_train, y_train)

Lamentablemente lanza un error :(

Prueba lo siguiente:

In [ ]:
ttr.fit(X_train.select_dtypes(include="number"), y_train)

__Pregunta:__ ¿Por qué falló el primer ajusto? ¿Qué tiene de diferente el segundo?

__Respuesta:__ Porque tiene columnas categóricas, a las que no se les puede aplicar log10, la segunda selecciona solo aquellas columnas que tienen valores numéricos y por eso no tira error.

## Ejercicio 4

(1 pto)

Ahora agreguemos todos los ingredientes a la juguera.

* Define `model` utilizando `make_pipeline` con los insumos `preprocessor` y `ttr`.
* Ajusta `model` con los datos de entrenamiento.
* Calcula el error absoluto medio con los datos de test.

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.metrics import median_absolute_error

model = make_pipeline(
    preprocessor,
    ttr
)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mae = median_absolute_error(y_pred, y_test)

print(f"El error absoluto medio obtenido es {mae}")